In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import matplotlib.pyplot as plt
import gc

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from lightgbm import early_stopping
from lightgbm import log_evaluation

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cafa-5-ohe-wm/trainMFO.csv
/kaggle/input/cafa-5-ohe-wm/trainBPO.csv
/kaggle/input/cafa-5-ohe-wm/trainCCO.csv
/kaggle/input/cafa-5-protein-function-prediction/sample_submission.tsv
/kaggle/input/cafa-5-protein-function-prediction/IA.txt
/kaggle/input/cafa-5-protein-function-prediction/Test (Targets)/testsuperset.fasta
/kaggle/input/cafa-5-protein-function-prediction/Test (Targets)/testsuperset-taxon-list.tsv
/kaggle/input/cafa-5-protein-function-prediction/Train/train_terms.tsv
/kaggle/input/cafa-5-protein-function-prediction/Train/train_sequences.fasta
/kaggle/input/cafa-5-protein-function-prediction/Train/train_taxonomy.tsv
/kaggle/input/cafa-5-protein-function-prediction/Train/go-basic.obo


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Analisando o banco de dados

In [2]:
def read_fasta_file(file_path):
    entries = []
    with open(file_path, "r") as fasta_file:
        lines = fasta_file.readlines()
        i = 0
        while i < len(lines):
            if lines[i].startswith(">"):
                entry = {}
                entry['EntryID'] = lines[i][1:].split()[0]
                
                #entry["OX"] = lines[i].split("OX=")[1].split()[0]
                i += 1
                sequence_lines = []
                while i < len(lines) and not lines[i].startswith(">"):
                    sequence_lines.append(lines[i].strip())
                    i += 1
                entry['seq'] = "".join(sequence_lines)
                entries.append(entry)
            else:
                i += 1
    df = pd.DataFrame(entries)
    df.set_index('EntryID', inplace=True)
    return df

In [3]:
# Entrada
trainFasta = read_fasta_file('/kaggle/input/cafa-5-protein-function-prediction/Train/train_sequences.fasta')

# Saída
trainTermsID = pd.read_csv('/kaggle/input/cafa-5-protein-function-prediction/Train/train_terms.tsv',
                           sep='\t')
# Teste
testFasta = read_fasta_file('/kaggle/input/cafa-5-protein-function-prediction/Test (Targets)/testsuperset.fasta')

print(trainTermsID['term'].value_counts(normalize=True)*100)
print(trainTermsID['term'].nunique())

GO:0005575    1.732184
GO:0008150    1.719097
GO:0110165    1.701870
GO:0003674    1.466052
GO:0005622    1.319665
                ...   
GO:0031772    0.000019
GO:0042324    0.000019
GO:0031771    0.000019
GO:0051041    0.000019
GO:0102628    0.000019
Name: term, Length: 31466, dtype: float64
31466


In [4]:
trainTermsID['aspect'].value_counts()

BPO    3497732
CCO    1196017
MFO     670114
Name: aspect, dtype: int64

In [5]:
trainFasta

,seq
EntryID,
P20536,MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIP...
O73864,MTEYRNFLLLFITSLSVIYPCTGISWLGLTINGSSVGWNQTHHCKL...
O95231,MRLSSSPPRGPQQLSSFGSVDWLSQSSCSGPTHTPRPADFSLGSLP...
A0A0B4J1F4,MGGEAGADGPRGRVKSLGLVFEDESKGCYSSGETVAGHVLLEAAEP...
P54366,MVETNSPPAGYTLKRSPSDLGEQQQPPRQISRSPGNTAAYHLTTAM...
...,...
A0A286YAI0,METEVDDFPGKASIFSQVNPLYSNNMKLCEAERYDFQHSEPKTMKS...
A0A1D5NUC4,MSAAASAEMIETPPVLNFEEIDYKEIEVEEVVGRGAFGVVCKAKWR...
Q5RGB0,MADKGPILTSVIIFYLSIGAAIFQILEEPNLNSAVDDYKNKTNNLL...


In [6]:
testFasta

,seq
EntryID,
Q9CQV8,MTMDKSELVQKAKLAEQAERYDDMAAAMKAVTEQGHELSNEERNLL...
P62259,MDDREDLVYQAKLAEQAERYDEMVESMKKVAGMDVELTVEERNLLS...
P68510,MGDREQLLQRARLAEQAERYDDMASAMKAVTELNEPLSNEDRNLLS...
P61982,MVDREQLVQKARLAEQAERYDDMAAAMKNVTELNEPLSNEERNLLS...
O70456,MERASLIQKAKLAEQAERYEDMAAFMKSAVEKGEELSCEERNLLSV...
...,...
P08380,GNCKCDDEGPNVRTAPLTGYVDLGYCNEGWEKCASYYSPIAECCRKKK
C0HK72,RGICLEPKVVGPCKARIRRFYYDSETGKCTPFIYGGCGGNGNNFET...
C0HK73,GSICLEPKVVGPCKAGIRRFYFDSETGKCTLFLYGGCKGNGNNFET...


In [7]:
trainTermsID

,EntryID,term,aspect
0,A0A009IHW8,GO:0008152,BPO
1,A0A009IHW8,GO:0034655,BPO
2,A0A009IHW8,GO:0072523,BPO
3,A0A009IHW8,GO:0044270,BPO
4,A0A009IHW8,GO:0006753,BPO
...,...,...,...
5363858,X5L565,GO:0050649,MFO
5363859,X5L565,GO:0016491,MFO
5363860,X5M5N0,GO:0005515,MFO
5363861,X5M5N0,GO:0005488,MFO


In [8]:
trainTermsID['EntryID'].value_counts().mean()

37.70835735275509

In [9]:
trainFull = trainTermsID.merge(trainFasta, on='EntryID', how='left')
#trainFull.drop('aspect', axis=1, inplace=True)
trainFull

,EntryID,term,aspect,seq
0,A0A009IHW8,GO:0008152,BPO,MSLEQKKGADIISKILQIQNSIGKTTSPSTLKTKLSEISRKEQENA...
1,A0A009IHW8,GO:0034655,BPO,MSLEQKKGADIISKILQIQNSIGKTTSPSTLKTKLSEISRKEQENA...
2,A0A009IHW8,GO:0072523,BPO,MSLEQKKGADIISKILQIQNSIGKTTSPSTLKTKLSEISRKEQENA...
3,A0A009IHW8,GO:0044270,BPO,MSLEQKKGADIISKILQIQNSIGKTTSPSTLKTKLSEISRKEQENA...
4,A0A009IHW8,GO:0006753,BPO,MSLEQKKGADIISKILQIQNSIGKTTSPSTLKTKLSEISRKEQENA...
...,...,...,...,...
5363858,X5L565,GO:0050649,MFO,MDLIPSFSTETWLLLAISLVLLYLYGTYTHGIFRKLGIPGPTPLPF...
5363859,X5L565,GO:0016491,MFO,MDLIPSFSTETWLLLAISLVLLYLYGTYTHGIFRKLGIPGPTPLPF...
5363860,X5M5N0,GO:0005515,MFO,MPDSITNGGRPPAPPSSVSSTTASTTGNFGTRRRLVNRIKKVDELH...
5363861,X5M5N0,GO:0005488,MFO,MPDSITNGGRPPAPPSSVSSTTASTTGNFGTRRRLVNRIKKVDELH...


In [10]:
def keep_most_frequent_outputs(df, percentage):
    # Calculate output frequency
    output_counts = df['term'].value_counts()

    # Calculate the number of classes to keep based on the percentage
    n = int(len(output_counts) * percentage)

    # Get the "n" most frequent outputs
    most_frequent_outputs = output_counts.head(n).index

    # Filter the DataFrame to keep rows with the most frequent outputs
    filtered_df = df[df['term'].isin(most_frequent_outputs)].copy()

    # Calculate the percentage of the database removed
    removed_percentage = (1 - len(filtered_df) / len(df)) * 100
    print("Percentage of database removed: {:.2f}%".format(removed_percentage))
    
    return filtered_df

trainFullD = keep_most_frequent_outputs(trainFull, 0.15)
trainFullD.drop('EntryID', axis=1, inplace=True)

Percentage of database removed: 6.64%


In [11]:
trainFullD

,term,aspect,seq
0,GO:0008152,BPO,MSLEQKKGADIISKILQIQNSIGKTTSPSTLKTKLSEISRKEQENA...
1,GO:0034655,BPO,MSLEQKKGADIISKILQIQNSIGKTTSPSTLKTKLSEISRKEQENA...
2,GO:0072523,BPO,MSLEQKKGADIISKILQIQNSIGKTTSPSTLKTKLSEISRKEQENA...
3,GO:0044270,BPO,MSLEQKKGADIISKILQIQNSIGKTTSPSTLKTKLSEISRKEQENA...
4,GO:0006753,BPO,MSLEQKKGADIISKILQIQNSIGKTTSPSTLKTKLSEISRKEQENA...
...,...,...,...
5363857,GO:0003674,MFO,MDLIPSFSTETWLLLAISLVLLYLYGTYTHGIFRKLGIPGPTPLPF...
5363859,GO:0016491,MFO,MDLIPSFSTETWLLLAISLVLLYLYGTYTHGIFRKLGIPGPTPLPF...
5363860,GO:0005515,MFO,MPDSITNGGRPPAPPSSVSSTTASTTGNFGTRRRLVNRIKKVDELH...
5363861,GO:0005488,MFO,MPDSITNGGRPPAPPSSVSSTTASTTGNFGTRRRLVNRIKKVDELH...


# Separando em 3 grupos

In [12]:
trainBPO = trainFullD[trainFullD['aspect'] == 'BPO'].copy()
trainBPO.drop('aspect', axis=1, inplace=True)
trainCCO = trainFullD[trainFullD['aspect'] == 'CCO'].copy()
trainCCO.drop('aspect', axis=1, inplace=True)
trainMFO = trainFullD[trainFullD['aspect'] == 'MFO'].copy()
trainMFO.drop('aspect', axis=1, inplace=True)

In [13]:
trainBPO = trainBPO.drop_duplicates().reset_index(drop=True)
trainCCO = trainCCO.drop_duplicates().reset_index(drop=True)
trainMFO = trainMFO.drop_duplicates().reset_index(drop=True)

In [14]:
print(trainMFO['term'].value_counts())
print(trainMFO['seq'].nunique())

GO:0003674    77632
GO:0005488    56588
GO:0005515    47330
GO:0003824    25141
GO:0097159    12785
              ...  
GO:0003954       81
GO:0019212       81
GO:0004016       81
GO:0016661       81
GO:0043394       81
Name: term, Length: 614, dtype: int64
77632


# One-Hot Encode

In [15]:
gc.collect()
trainBPOe = pd.get_dummies(trainBPO, columns=['term'], sparse=True,
                           prefix='', prefix_sep='')
gc.collect()
trainCCOe = pd.get_dummies(trainCCO, columns=['term'], sparse=True,
                           prefix='', prefix_sep='')
gc.collect()
trainMFOe = pd.get_dummies(trainMFO, columns=['term'], sparse=True,
                           prefix='', prefix_sep='')
gc.collect()
trainMFOe

,seq,GO:0000030,GO:0000049,GO:0000146,GO:0000149,GO:0000166,GO:0000175,GO:0000217,GO:0000287,GO:0000976,...,GO:1901681,GO:1901682,GO:1901702,GO:1901981,GO:1902936,GO:1904315,GO:1904680,GO:1990782,GO:1990837,GO:1990841
0,MSLEQKKGADIISKILQIQNSIGKTTSPSTLKTKLSEISRKEQENA...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,MSLEQKKGADIISKILQIQNSIGKTTSPSTLKTKLSEISRKEQENA...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,MSLEQKKGADIISKILQIQNSIGKTTSPSTLKTKLSEISRKEQENA...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,MSLEQKKGADIISKILQIQNSIGKTTSPSTLKTKLSEISRKEQENA...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,MSLEQKKGADIISKILQIQNSIGKTTSPSTLKTKLSEISRKEQENA...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600653,MDLIPSFSTETWLLLAISLVLLYLYGTYTHGIFRKLGIPGPTPLPF...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
600654,MDLIPSFSTETWLLLAISLVLLYLYGTYTHGIFRKLGIPGPTPLPF...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
600655,MPDSITNGGRPPAPPSSVSSTTASTTGNFGTRRRLVNRIKKVDELH...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
600656,MPDSITNGGRPPAPPSSVSSTTASTTGNFGTRRRLVNRIKKVDELH...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
def process_dataframe(df):
    df = df.set_index('seq')  # Set 'seq' column as the index
    unique_seqs = df.index.unique()
    unique_classes = df.columns  # Assuming class columns start from index 0
    new_df = pd.DataFrame(0, index=unique_seqs, columns=unique_classes)

    progress_bar = tqdm(total=len(unique_classes), desc="Processing Columns")

    for col in unique_classes:
        gc.collect()
        class_indices = df[df[col] == 1].index
        new_df.loc[class_indices, col] = 1

        progress_bar.update(1)

    progress_bar.close()
    return new_df.reset_index()


trainBPOenc = process_dataframe(trainBPOe)
trainCCOenc = process_dataframe(trainCCOe)
trainMFOenc = process_dataframe(trainMFOe)
trainMFOenc.describe()

Processing Columns:  37%|███▋      | 1312/3564 [11:46<33:20,  1.13it/s] 

In [ ]:
trainBPOenc

In [ ]:
trainCCOenc

In [ ]:
trainMFOenc

In [ ]:
trainBPOenc.to_csv('trainBPO.csv', index=False)
trainCCOenc.to_csv('trainCCO.csv', index=False)
trainMFOenc.to_csv('trainMFO.csv', index=False)